In [1]:
# LangChain Tools
# 외부 기능이나  api 
# 주식, 날시, 웹 검색 ....
# llm 훈련데이터 이후의 정보는 모른다 - 웹 검색
# 외부 API 호출 Tool 구현

In [ ]:
%pip install langchain-core langchain-openai openai python-dotenv requests

In [3]:
from dotenv import load_dotenv
import os

load_dotenv()
print(os.getenv('OPENAI_API_KEY')[:5])
print(os.getenv('OWM_API_KEY')[:5])

sk-pr
ff3b4


In [16]:
import requests
from pydantic import BaseModel, Field

class WeatherInput(BaseModel):    
    city : str = Field(description="날씨를 조회할 도시 이름(영문)")

def get_weather(city: str) -> str:
    '''주어진 도시의 현재 날씨를 반환합니다'''
    api_key = os.getenv('OWM_API_KEY')
    url = 'https://api.openweathermap.org/data/2.5/weather'
    city = 'Seoul'
    params = {
        'q':city,
        'appid':api_key,
        'units':'metric',  # 섭씨온도
        'lang' : 'kr'
    }
    response = requests.get(url,params=params)
    data = response.json()
    if data.get('cod') != 200:
        print(f'{city}의 날씨 정보를 찾을 수 없습니다.')
    # 필요한 정보를 파싱
    temp = data['main']['temp']
    desc = data['weather'][0]['description']
    city_name = data['name']
    return f'{city_name}의 현재 기온은 {temp}c , 날씨는 {desc} 입니다.' 

In [17]:
print(get_weather('Seoul'))

Seoul의 현재 기온은 33.76c , 날씨는 구름조금 입니다.


In [33]:
import requests
from dotenv import load_dotenv
import os
load_dotenv()

class NewsInput(BaseModel):
    keyword: str = Field(description="최신뉴스를 검색할 키워드(한글 또는 영문)")

def get_news(keyword:str) ->str:
    url = f"https://openapi.naver.com/v1/search/news.json?query={keyword}&display=5&sort=date"

    # 헤더 설정
    headers = {
        "X-Naver-Client-Id": os.getenv("NAVER_CLIENT_ID"),
        "X-Naver-Client-Secret": os.getenv("NAVER_CLIENT_SECRET")
    }

    # API 요청
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        article = data['items'][0]
        title = article['title']
        source = article['description']
        return f'{keyword} 관련뉴스:{title} - {source}'

In [36]:
get_news('인공지능')

"인공지능 관련뉴스:반성문까지 썼지만 구체성 부족…단기 성장률 제고 처방도 필요하다 - 정부는 AI(<b>인공지능</b>) 대전환 등을 통해 잠재성장률을 끌어올리겠다는 계획이지만 일각에선 새정부의 'AI 만능주의'를 우려하는 시각도 있다. 문재인정부의 '수소경제'처럼 특정 산업에 '묻지마식 올인'을 하는 건... "

In [39]:
# 기능을 정의한 두 개의  함수를 langchain의  tool로 등록
from langchain_core.runnables import RunnableLambda
weather_runable = RunnableLambda(lambda input: get_weather(input['city']))
weather_tool = weather_runable.as_tool(
    name='get_weather',
    description='도시이름을 입력하면 현재 날씨를 알려주는 도구',
    args_schema=WeatherInput
)

news_runable = RunnableLambda(lambda input: get_news(input['keyword']))
news_tool = news_runable.as_tool(
    name='get_news',
    description=' 키워드를 입력하면 최신  뉴스 제목을 알려주는 도구',
    args_schema=NewsInput
)

C:\Users\tj\AppData\Local\Temp\ipykernel_1860\3518708658.py:4: LangChainBetaWarning: This API is in beta and may change in the future.
  weather_tool = weather_runable.as_tool(


In [43]:
# 에이전트 생성 및 툴 등록
from langchain.agents import initialize_agent, AgentType
from langchain_openai import ChatOpenAI

# 앞에서 만든 툴 객체 리스트 준비
tools = [weather_tool, news_tool]
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)
# openai기반 에이전트 생성
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True
)

C:\Users\tj\AppData\Local\Temp\ipykernel_1860\3379685680.py:9: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [44]:
query1 = '뉴욕(New York)의 날씨가 어떤지 알려줘'
result1 =  agent.run(query1)
print(result1)

C:\Users\tj\AppData\Local\Temp\ipykernel_1860\1394134721.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result1 =  agent.run(query1)




> Entering new AgentExecutor chain...

Invoking: `get_weather` with `{'city': 'New York'}`


Seoul의 현재 기온은 32.22c , 날씨는 튼구름 입니다.현재 뉴욕의 날씨는 흐림이며, 기온은 약 32.22도입니다.

> Finished chain.
현재 뉴욕의 날씨는 흐림이며, 기온은 약 32.22도입니다.


In [45]:
query2 = '서울의 날씨와 한일관계에 대한 최신 뉴스 알려줘'
result2 = agent.run(query2)
print(result2)



> Entering new AgentExecutor chain...

Invoking: `get_weather` with `{'city': 'Seoul'}`


Seoul의 현재 기온은 34.76c , 날씨는 약간의 구름이 낀 하늘 입니다.
Invoking: `get_news` with `{'keyword': '한일관계'}`


한일관계 관련뉴스:이 대통령 '셔틀 외교' 일본 일정 마무리…워싱턴으로 출발 - 특히 이 대통령은 회담에서 &quot;국제질서가 요동치고 있다&quot;며 &quot;가치·질서·체제·이념에서 비슷한 입장을 가진 <b>한일</b> 양국이 어느 때보다 협력 <b>관계</b>를 강화해야 한다&quot;고 강조했습니다. 한·미 정상회담을 앞두고 한·일<b>관계</b>... 현재 서울의 날씨는 약간의 구름이 낀 하늘이며, 기온은 34.76도입니다.

한일관계에 대한 최신 뉴스로는 이 대통령이 일본 일정을 마무리하고 워싱턴으로 출발했다는 소식이 있습니다. 이 대통령은 회담에서 "국제질서가 요동치고 있다"며 "가치·질서·체제·이념에서 비슷한 입장을 가진 한일 양국이 어느 때보다 협력 관계를 강화해야 한다"고 강조했습니다.

> Finished chain.
현재 서울의 날씨는 약간의 구름이 낀 하늘이며, 기온은 34.76도입니다.

한일관계에 대한 최신 뉴스로는 이 대통령이 일본 일정을 마무리하고 워싱턴으로 출발했다는 소식이 있습니다. 이 대통령은 회담에서 "국제질서가 요동치고 있다"며 "가치·질서·체제·이념에서 비슷한 입장을 가진 한일 양국이 어느 때보다 협력 관계를 강화해야 한다"고 강조했습니다.


In [ ]:
# 랭그래프
%pip install langgraph

In [58]:
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage

In [62]:
agent = create_react_agent(llm,tools)
query = '서울의 날씨와 한일관계에 대한 최신 뉴스 알려줘'
result = agent.invoke({
    'messages' : [HumanMessage(content=query)]
})
print(result['messages'][-1].content)

서울의 현재 기온은 34.76도이며, 날씨는 맑습니다.

한일관계에 대한 최신 뉴스는 다음과 같습니다:
- 최근 한일관계에 대한 논의가 활발히 이루어지고 있으며, 특히 일본의 경제 정책과 관련된 내용이 주목받고 있습니다. 일본의 경제 정책이 한국에 미치는 영향과 관련하여 다양한 의견이 제시되고 있습니다. 

더 구체적인 내용이나 추가적인 정보가 필요하시면 말씀해 주세요!
